In [ ]:
pip install langchain==0.2

In [ ]:
pip install dsm-services

In [ ]:
import dsm_services
import pandas as pd
from dsm_services.llm.langchain import llmChatModel
from tqdm.auto import tqdm

In [ ]:
tqdm.pandas()

In [ ]:
model = dsm_services.llm.LLM(
    model_name='llama3.1',
    api_key="H9hWk3vo.iWZcjb3Kg8Cq57j2iT0QtjJG8LDbqwbU"
)
llm = llmChatModel(
    model_name="llama3.1:70b",
    api_key="H9hWk3vo.iWZcjb3Kg8Cq57j2iT0QtjJG8LDbqwbU"
)

In [ ]:
from typing import List, Dict

from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator

from langchain.prompts import PromptTemplate
from langchain.schema import BaseOutputParser
from pydantic import BaseModel, ValidationError
from typing import Any, List

In [ ]:
import time
from functools import wraps

def retry(n):
    """
    A decorator to retry a function n times upon failure, and print the number of attempts made.

    Parameters:
    -----------
    n : int
        The number of retry attempts.

    Returns:
    --------
    Callable
        A decorator that retries the decorated function up to n times.
    """
    def decorator_retry(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < n:
                try:
                    attempts += 1
                    # Pass the attempt count to the decorated function
                    result = func(*args, attempt_number=attempts, **kwargs)
                    print(f"Success on attempt {attempts}")
                    return result
                except Exception as e:
                    print(f"Attempt {attempts} failed: {e}")
                    if attempts == n:
                        raise  # Re-raise the exception if it's the last attempt
                    # time.sleep(2)  # Optional delay between retries
        return wrapper
    return decorator_retry

In [ ]:
class Entity(BaseModel):
    """
    An entity details in the knowledge graph. The entity detials consist of name, class and description.

    Attributes:
    -----------
    name : str
        The human-readable name of the entity.
    class : str
        The classification or type of the entity (e.g., Person, Product, etc.).
    description : str
        A brief description of the entity.
    """
    entity_name: str
    entity_class: str
    description: str

class Relation(BaseModel):
    """
    A class to represent a relationship (triple) in the knowledge graph.

    Attributes:
    -----------
    head : Entity
        The subject or head entity in the relationship.
    predicate : str
        The relationship or predicate that links the head and tail (e.g., Person, Product, etc.).
    tail : Entity
        The object or tail entity in the relationship.
    """
    head: Entity
    predicate: str
    tail: Entity

In [ ]:
# Define a custom Output Parser
# class EntityRelationOutputParser(BaseOutputParser):
#     def parse(self, text: str) -> List[Relation]:
#         try:
#             # You can implement custom parsing logic here, for example, splitting or extracting structured data
#             parsed_data = eval(text)  # Assuming LLM responds in Python dictionary format
#             relations = [Relation(**relation) for relation in parsed_data]
#             return relations
#         except ValidationError as e:
#             raise ValueError(f"Parsing error: {e}")

class EntityRelationOutputParser(BaseOutputParser):
    def parse(self, text: str) -> Dict[str, pd.DataFrame]:
        try:
            # Parse the response assuming it comes in JSON format
            parsed_data = eval(text)  # Assuming LLM responds in Python dictionary format

            # Extract relations
            relations = [Relation(**relation) for relation in parsed_data]

            # Create unique entity records for head and tail
            entities = {}
            for relation in relations:
                entities[relation.head.entity_name] = relation.head
                entities[relation.tail.entity_name] = relation.tail

            # Create DataFrames
            entity_df = pd.DataFrame([entity.dict() for entity in entities.values()])
            relations_df = pd.DataFrame([{
                'head_name': relation.head.entity_name,
                'head_class': relation.head.entity_class,
                'head_description': relation.head.description,
                'predicate': relation.predicate,
                'tail_name': relation.tail.entity_name,
                'tail_class': relation.tail.entity_class,
                'tail_description': relation.tail.description
            } for relation in relations])

            return {
                "entity_df": entity_df,
                "relations_df": relations_df
            }

        except ValidationError as e:
            raise ValueError(f"Parsing error: {e}")

# Define your prompt template
prompt_template = """
Please extract the entities and relations from the following Wikipedia text to the database.
Ensure that the entities and relationships are accurate.
Given the following input, return the entities and relations as JSON format.

ps. reponse only json data

Context: {context}

Input: {input_text}

entity_class may be are: [Person, Organization, Product, Time, Location]

Format your output as follows:
[
    {{
        "head": {{
            "entity_name": "name of the head entity",
            "entity_class": "classification of head",
            "description": "brief description"
        }},
        "predicate": "relationship description",
        "tail": {{
            "entity_name": "name of the tail entity",
            "entity_class": "classification of tail",
            "description": "brief description"
        }}
    }}
]

Response:
"""

# Instantiate a PromptTemplate
prompt = PromptTemplate(
    input_variables=["input_text", "context"],
    template=prompt_template
)

# Function to create the full prompt and parse output
@retry(5)
def create_and_parse_output(input_text: str, context:str, llm, attempt_number) -> List[Relation]:
    # Create the full prompt
    final_prompt = prompt.format(input_text=input_text, context=context)

    # Call the LLM (e.g., OpenAI, Ollama) with the prompt
    response = llm.invoke(final_prompt).content  # This depends on your LLM client

    # Parse the output using the custom parser
    parser = EntityRelationOutputParser()
    output = parser.parse(response)
    output.update({
        'attempt_number': attempt_number
    })
    return output

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
df = pd.read_csv('/content/news.csv')

In [ ]:
print(df.columns)
df

Index(['Date', 'Url', 'Content '], dtype='object')


,Date,Url,Content
0,1/8/2011,https://www.thaipbs.or.th/news/content/26028,สถานการณ์น้ำท่วมหลายพื้นที่ในภาคเหนือยังน่าเป็...
1,2/8/2011,https://www.thaipbs.or.th/news/content/26139,ฝนที่ตกต่อเนื่องในภาคเหนือทำให้ประชาชนยังต้องเ...
2,31/8/2011,https://www.thaipbs.or.th/news/content/31210,สำนักงานป้องกันและบรรเทาสาธารณภัยจังหวัด เตือน...
3,20/8/2011,https://www.thaipbs.or.th/news/content/29222,ชาวจังหวัดสุโขทัยบางส่วนแต่งชุดดำร่วมไว้อาลัยใ...
4,15/9/2011,https://www.thaipbs.or.th/news/content/33933,ขณะที่ประชาชนกำลังเผชิญกับสถานการณ์น้ำท่วมที่ย...
...,...,...,...
302,6/2/2012,https://www.thaipbs.or.th/news/content/64288,วิกฤตน้ำท่วมยังส่งผลกระทบต่อภาคธุรกิจ ล่าสุดกร...
303,6/2/2012,https://www.thaipbs.or.th/news/content/64289,กรุงเทพมหานครถูกตัดงบประมาณบริหารจัดการน้ำท่วม...
304,8/2/2012,https://www.thaipbs.or.th/news/content/64804,การฟื้นฟูนิคมอุตสาหกรรมทั้ง 7 แห่ง หลังเผชิญกั...
305,10/2/2012,https://www.thaipbs.or.th/news/content/65427,นายกฯกล่าวเปิดงานรักเมืองไทยฯขอบคุณทุกหน่วยงาน...


In [ ]:
content = df['Content '].iloc[65]

In [ ]:
text = content
text

'น้ำที่เอ่อล้นจากคลองประปาได้ไหลเข้าท่วมพื้นที่ตลอดแนวถนนนาวงประชาพัฒนา ในแขวงทุ่งสีกันเขตดอนเมืองตั้งแต่เมื่อคืนที่ผ่านมา ทำให้ประชาชนในหมู่บ้านปิ่นเจริญ 4 กว่า 700 หลังคาเรือน ต้องถูกน้ำท่วมสูงกว่า 50 เซนติเมตร ประชาชนต้องเร่งอพยพออกนอกพื้นที่ และ จนถึงขณะนี้ยังไม่มีหน่วยงานใดเข้าไปช่วยเหลือ\nน้ำจากคลองประปาไหลเข้าท่วมพื้นที่ถนนนาวงประชาพัฒนา แขวงทุ่งสีกัน เขตดอนเมือง ตั้งแต่เมื่อคืนที่ผ่านมา ทำให้ประชาชนในหมู่บ้านปิ่นเจริญ 4 ที่มีกว่า 700 หลังคาเรือน ต้องเร่งขนของขึ้นที่สูง ขณะที่บางคนก็อพยพออกนอกพื้นที่ไปพักอาศัยบ้านญาติพี่น้อง เนื่องจากระดับน้ำภายในหมู่บ้านขณะนี้สูงกว่า 50 เซนติเมตร\n\nชาวบ้าน บอกว่า เหตุการณ์ครั้งนี้ไม่เคยเกิดขึ้นมาก่อน โดยน้ำได้เข้าท่วมหมู่บ้านตั้งแต่ช่วงค่ำของวันที่ 20 ตุลาคม สภาพที่ไหลแรงและเพิ่มขึ้นอย่างรวดเร็ว ทำให้ไม่สามารถตั้งตัวได้ทัน และในขณะนี้ก็ยังไม่มีหน่วยงานใดเข้ามาให้การช่วยเหลือ\n\nน้ำที่ไหลมาจากคลองประปาจะไหลข้ามถนนนาวงประชาพัฒนา ถนนเทิดราชันย์ ไปสมทบกันที่คลองนายกิม ก่อนจะไหลลงสู่คลองเปรมประชากร และตลอดเส้นทางของน้ำยังมีชุมชน หน่วยงานราชการ ที่ไ

In [ ]:
output = create_and_parse_output(
    llm=llm,
    input_text=text,
    context="น้ำท่วม"
)
output

Success on attempt 1


{'entity_df':                    entity_name  entity_class  \
 0                    คลองประปา      Location   
 1            ถนนนาวงประชาพัฒนา      Location   
 2          หมู่บ้านปิ่นเจริญ 4      Location   
 3  ประชาชนกว่า 700 หลังคาเรือน  Organization   
 4                แขวงทุ่งสีกัน      Location   
 5                กรุงเทพมหานคร      Location   
 6                 ถนนเทิดราชัน      Location   
 7             วันที่ 20 ตุลาคม          Time   
 8   น้ำท่วมหมู่บ้านปิ่นเจริญ 4         Event   
 
                                          description  
 0  คลองประปาไหลข้ามถนนนาวงประชาพัฒนาและถนนเทิดราช...  
 1        ถนนนาวงประชาพัฒนาต้องรับน้ำท่วมจากคลองประปา  
 2     หมู่บ้านปิ่นเจริญ 4 ต้องรับน้ำท่วมจากคลองประปา  
 3  ประชาชนที่อาศัยอยู่ในหมู่บ้านปิ่นเจริญ 4 ต้องร...  
 4              แขวงทุ่งสีกันเป็นพื้นที่เสี่ยงน้ำท่วม  
 5  กรุงเทพมหานครประกาศแขวงทุ่งสีกันเป็นพื้นที่เสี...  
 6   ถนนเทิดราชันเป็นเส้นทางที่น้ำจากคลองประปาไหลผ่าน  
 7    เหตุการณ์น้ำท่วมเกิดขึ้นตั้งแต่วันที่ 20 

In [ ]:
entity_df = output['entity_df']
entity_df

,entity_name,entity_class,description
0,คลองประปา,Location,คลองประปาไหลข้ามถนนนาวงประชาพัฒนาและถนนเทิดราช...
1,ถนนนาวงประชาพัฒนา,Location,ถนนนาวงประชาพัฒนาต้องรับน้ำท่วมจากคลองประปา
2,หมู่บ้านปิ่นเจริญ 4,Location,หมู่บ้านปิ่นเจริญ 4 ต้องรับน้ำท่วมจากคลองประปา
3,ประชาชนกว่า 700 หลังคาเรือน,Organization,ประชาชนที่อาศัยอยู่ในหมู่บ้านปิ่นเจริญ 4 ต้องร...
4,แขวงทุ่งสีกัน,Location,แขวงทุ่งสีกันเป็นพื้นที่เสี่ยงน้ำท่วม
5,กรุงเทพมหานคร,Location,กรุงเทพมหานครประกาศแขวงทุ่งสีกันเป็นพื้นที่เสี...
6,ถนนเทิดราชัน,Location,ถนนเทิดราชันเป็นเส้นทางที่น้ำจากคลองประปาไหลผ่าน
7,วันที่ 20 ตุลาคม,Time,เหตุการณ์น้ำท่วมเกิดขึ้นตั้งแต่วันที่ 20 ตุลาคม
8,น้ำท่วมหมู่บ้านปิ่นเจริญ 4,Event,เหตุการณ์น้ำท่วมหมู่บ้านปิ่นเจริญ 4 เกิดขึ้นตั...


In [ ]:
relations_df = output['relations_df'][['head_name', 'predicate', 'tail_name']]
relations_df

,head_name,predicate,tail_name
0,คลองประปา,ไหลเข้าท่วม,ถนนนาวงประชาพัฒนา
1,หมู่บ้านปิ่นเจริญ 4,มีประชากร,ประชาชนกว่า 700 หลังคาเรือน
2,แขวงทุ่งสีกัน,เป็นส่วนหนึ่งของ,กรุงเทพมหานคร
3,คลองประปา,ไหลข้าม,ถนนเทิดราชัน
4,วันที่ 20 ตุลาคม,เป็นวันที่เกิดเหตุการณ์,น้ำท่วมหมู่บ้านปิ่นเจริญ 4


In [ ]:
data = relations_df\
    .merge(right=entity_df, left_on='head_name', right_on='entity_name', how='left')\
    .merge(right=entity_df, left_on='tail_name', right_on='entity_name', how='left')

In [ ]:
data['index'] = '67'
data

,head_name,predicate,tail_name,entity_name_x,entity_class_x,description_x,entity_name_y,entity_class_y,description_y,index
0,คลองประปา,ไหลเข้าท่วม,ถนนนาวงประชาพัฒนา,คลองประปา,Location,คลองประปาไหลข้ามถนนนาวงประชาพัฒนาและถนนเทิดราช...,ถนนนาวงประชาพัฒนา,Location,ถนนนาวงประชาพัฒนาต้องรับน้ำท่วมจากคลองประปา,67
1,หมู่บ้านปิ่นเจริญ 4,มีประชากร,ประชาชนกว่า 700 หลังคาเรือน,หมู่บ้านปิ่นเจริญ 4,Location,หมู่บ้านปิ่นเจริญ 4 ต้องรับน้ำท่วมจากคลองประปา,ประชาชนกว่า 700 หลังคาเรือน,Organization,ประชาชนที่อาศัยอยู่ในหมู่บ้านปิ่นเจริญ 4 ต้องร...,67
2,แขวงทุ่งสีกัน,เป็นส่วนหนึ่งของ,กรุงเทพมหานคร,แขวงทุ่งสีกัน,Location,แขวงทุ่งสีกันเป็นพื้นที่เสี่ยงน้ำท่วม,กรุงเทพมหานคร,Location,กรุงเทพมหานครประกาศแขวงทุ่งสีกันเป็นพื้นที่เสี...,67
3,คลองประปา,ไหลข้าม,ถนนเทิดราชัน,คลองประปา,Location,คลองประปาไหลข้ามถนนนาวงประชาพัฒนาและถนนเทิดราช...,ถนนเทิดราชัน,Location,ถนนเทิดราชันเป็นเส้นทางที่น้ำจากคลองประปาไหลผ่าน,67
4,วันที่ 20 ตุลาคม,เป็นวันที่เกิดเหตุการณ์,น้ำท่วมหมู่บ้านปิ่นเจริญ 4,วันที่ 20 ตุลาคม,Time,เหตุการณ์น้ำท่วมเกิดขึ้นตั้งแต่วันที่ 20 ตุลาคม,น้ำท่วมหมู่บ้านปิ่นเจริญ 4,Event,เหตุการณ์น้ำท่วมหมู่บ้านปิ่นเจริญ 4 เกิดขึ้นตั...,67


In [ ]:
# Set chunk size to process 50 rows at a time
chunk_size = 10
num_rows = len(df['Content '])

# Initialize lists to store results for all chunks
entity_dfs = []
relation_dfs = []

# Loop over the DataFrame in chunks of 50 rows
for start_idx in range(0, num_rows, chunk_size):
    end_idx = min(start_idx + chunk_size, num_rows)  # Ensure not to exceed total rows
    print(f"Processing rows {start_idx + 1} to {end_idx}")

    # Process each chunk
    for idx, content in enumerate(df['Content '].iloc[start_idx:end_idx]):
        print(f"  Processing row {start_idx + idx + 1}")

        # Extract entities and relations for the current row
        output = create_and_parse_output(
            llm=llm,
            input_text=content,
            context="น้ำท่วม"
        )

        # Add a "row" column to each DataFrame to indicate the original row
        output['entity_df']['row'] = start_idx + idx
        output['relations_df']['row'] = start_idx + idx

        # Append results to the lists
        entity_dfs.append(output['entity_df'])
        relation_dfs.append(output['relations_df'])

    # Optionally, save intermediate results to CSV after each chunk
    intermediate_entity_df = pd.concat(entity_dfs, ignore_index=True)
    intermediate_relation_df = pd.concat(relation_dfs, ignore_index=True)
    intermediate_entity_df.to_csv(f"intermediate_entities_rows_{start_idx+1}_to_{end_idx}.csv", index=False)
    intermediate_relation_df.to_csv(f"intermediate_relations_rows_{start_idx+1}_to_{end_idx}.csv", index=False)

# Combine all entity and relation DataFrames after processing all chunks
combined_entity_df = pd.concat(entity_dfs, ignore_index=True)
combined_relation_df = pd.concat(relation_dfs, ignore_index=True)

# Final merged DataFrame for all processed chunks
data = combined_relation_df \
    .merge(right=combined_entity_df, left_on='head_name', right_on='entity_name', how='left') \
    .merge(right=combined_entity_df, left_on='tail_name', right_on='entity_name', how='left')

# Save the final combined result to CSV
data.to_csv("final_extracted_relations_with_row.csv", index=False)

data


Processing rows 201 to 210
  Processing row 201
Attempt 1 failed: chat  Some thing wrong 504, <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
<hr><center>nginx/1.25.2</center>
</body>
</html>

Attempt 2 failed: chat  Some thing wrong 504, <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
<hr><center>nginx/1.25.2</center>
</body>
</html>

Attempt 3 failed: chat  Some thing wrong 504, <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
<hr><center>nginx/1.25.2</center>
</body>
</html>

Success on attempt 4
  Processing row 202
Attempt 1 failed: chat  Some thing wrong 504, <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
<hr><center>nginx/1.25.2</center>
</body>
</html>

Success on attempt 2
  Processing row 203
Success on attempt 1
  Processing row 20

,head_name,head_class,head_description,predicate,tail_name,tail_class,tail_description,row_x,entity_name_x,entity_class_x,description_x,row_y,entity_name_y,entity_class_y,description_y,row
0,ม.ร.ว.สุขุมพันธุ์ บริพัตร,Person,ผู้ว่าราชการกรุงเทพมหานคร,เป็นผู้ว่าราชการของ,กรุงเทพมหานคร,Location,เมืองหลวงของประเทศไทย,200,ม.ร.ว.สุขุมพันธุ์ บริพัตร,Person,ผู้ว่าราชการกรุงเทพมหานคร,200,กรุงเทพมหานคร,Location,เมืองหลวงของประเทศไทย,200
1,ม.ร.ว.สุขุมพันธุ์ บริพัตร,Person,ผู้ว่าราชการกรุงเทพมหานคร,เป็นผู้ว่าราชการของ,กรุงเทพมหานคร,Location,เมืองหลวงของประเทศไทย,200,ม.ร.ว.สุขุมพันธุ์ บริพัตร,Person,ผู้ว่าราชการกรุงเทพมหานคร,200,กรุงเทพมหานคร,Location,,208
2,ม.ร.ว.สุขุมพันธุ์ บริพัตร,Person,ผู้ว่าราชการกรุงเทพมหานคร,เป็นผู้ว่าราชการของ,กรุงเทพมหานคร,Location,เมืองหลวงของประเทศไทย,200,ม.ร.ว.สุขุมพันธุ์ บริพัตร,Person,ผู้ว่าราชการกรุงเทพมหานคร,200,กรุงเทพมหานคร,Location,capital city of Thailand,210
3,ม.ร.ว.สุขุมพันธุ์ บริพัตร,Person,ผู้ว่าราชการกรุงเทพมหานคร,เป็นผู้ว่าราชการของ,กรุงเทพมหานคร,Location,เมืองหลวงของประเทศไทย,200,ม.ร.ว.สุขุมพันธุ์ บริพัตร,Person,ผู้ว่าราชการกรุงเทพมหานคร,200,กรุงเทพมหานคร,Location,,211
4,ม.ร.ว.สุขุมพันธุ์ บริพัตร,Person,ผู้ว่าราชการกรุงเทพมหานคร,เป็นผู้ว่าราชการของ,กรุงเทพมหานคร,Location,เมืองหลวงของประเทศไทย,200,ม.ร.ว.สุขุมพันธุ์ บริพัตร,Person,ผู้ว่าราชการกรุงเทพมหานคร,200,กรุงเทพมหานคร,Location,,217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6454,ศูยัตรณ คุ่งาม,Person,อ่างัตรณ,นิควัม อ่าง,เป็นส์ คุ่งาม,Organization,อ่างัตรณ,306,ศูยัตรณ คุ่งาม,Person,อ่างัตรณ,306,เป็นส์ คุ่งาม,Organization,อ่างัตรณ,306
6455,ศูยัตรณ คุ่งาม,Person,อ่างัตรณ,เป็นส์,ย้าง ใครัตมเป็น,Location,อ่างัตรณ,306,ศูยัตรณ คุ่งาม,Person,อ่างัตรณ,306,ย้าง ใครัตมเป็น,Location,อ่างัตรณ,306
6456,เป็นส์,Event,อ่างัตรณ,เป็นส์,ศูยัตรณ คุ่งาม,Person,อ่างัตรณ,306,เป็นส์,Event,อ่างัตรณ,306,ศูยัตรณ คุ่งาม,Person,อ่างัตรณ,306
6457,เป็นส์,Event,อ่างัตรณ,นิควัม อ่าง,ย้าง ใครัตมเป็น,Location,อ่างัตรณ,306,เป็นส์,Event,อ่างัตรณ,306,ย้าง ใครัตมเป็น,Location,อ่างัตรณ,306
